In [80]:
import dhlab as dh
import pandas as pd
import dhlab.graph_networkx_louvain as gnl
import networkx as nx
from IPython.display import HTML

In [5]:
import igraph as ig
import leidenalg as la

from igraph import Graph
from leidenalg import find_partition
import numpy as np
import os

In [7]:
import conc_coll_corpus as co
import polnet_graphs as polg

In [8]:
from importlib import reload

In [12]:
zotero = co.zot()

In [32]:
G = polg.make_nx_graph(zotero)

## convert to igraph and use leiden algorithm

In [91]:
reload(polg)

<module 'polnet_graphs' from '/mnt/disk1/Github/polnet_plotly/polnet_graphs.py'>

In [124]:
h = polg.igraph_from_networkx(G)

In [153]:
degrees = polg.degree(h).set_index('vertex_id').to_dict()['degree']

In [155]:
for v in h.vs:
    v['centrality'] = degrees.get(v['name'], 0)

In [154]:
#dict([(x[0], x[1]) for  x in polg.degree(h).set_index('vertex_id').to_records()])

In [97]:
parts = polg.make_clusters(h)

In [126]:
h.vs['community'] = parts.membership

In [111]:
#[vertex for vertex in h.vs]

In [156]:
import json

# Assuming 'h' is your igraph graph

# Extract nodes
nodes = [{"id": vertex['name'], "label": vertex['label'], 'community': vertex['community'], 'centrality':int(vertex['centrality'])} for vertex in h.vs]

# Extract edges
links = [{"source": edge.source_vertex['name'], "target": edge.target_vertex['name']} for edge in h.es]

# Create a graph dictionary
graph = {"nodes": nodes, "links": links}

# Convert to JSON (if needed)
graph_json = json.dumps(graph)

# 'graph_json' is now ready to be used with D3.js


In [162]:
with open('polnet_test.html', "w", encoding = "utf-8") as fp:
    fp.write(f"""
<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8">
      <title>D3 Force Layout</title>
      <style>
         text {{
         font-family: sans-serif;
         }}
      </style>
   </head>
   <body>
      <svg width="1800" height="1600"></svg>
      <script src="https://d3js.org/d3.v5.min.js"></script>
      <script>
         // Load the network graph
          var graph = JSON.parse('{graph_json}'); 

         
         d3.select("svg").selectAll("*").remove();
     
         var nodes = graph.nodes;
         var links = graph.links;

         var svg = d3.select("svg"),
             width = +svg.attr("width"),
             height = +svg.attr("height");

        var centralityScale = d3.scaleLinear()
         .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
         .range([5, 35]);  // Min and max circle radii
         
        var fontSizeScale = d3.scaleLinear()
        .domain([d3.min(nodes, d => d.centrality), d3.max(nodes, d => d.centrality)])
        .range([10, 50]);  // Min and max font sizes, adjust as needed
        
        var opacityScale = d3.scaleLinear()
        .domain([d3.max(nodes, d => d.centrality), d3.min(nodes, d => d.centrality)])
        .range([0.2, 0.7]);  // Min and max font sizes, adjust as needed

         var simulation = d3.forceSimulation(nodes)
             .force("link", d3.forceLink(links).id(d => d.id))
             .force("charge", d3.forceManyBody().strength(-150))
             .force("center", d3.forceCenter(width / 2, height / 2));

                 
         // Add links
         var link = svg.append("g")
         .attr("class", "links")
         .selectAll("line")
         .data(links)
         .enter().append("line")
         .attr("stroke", "#999")
         .attr("stroke-opacity", 0.1);
         
         // Add nodes
         // Create groups for nodes and labels
         var node = svg.append("g")
         .attr("class", "nodes")
         .selectAll(".node")
         .data(nodes)
         .enter().append("g")
         .attr("class", "node")
         .call(d3.drag()
         .on("start", dragstarted)
         .on("drag", dragged)
         .on("end", dragended));
         
         // Append circles to the groups
         var color = d3.scaleOrdinal(d3.schemeCategory10);  // adjust if needed
         
         // When appending circles:
         node.append("circle")
         .attr("r", d => centralityScale(d.centrality))
         .attr("fill", d => color(d.community))
         .attr("fill-opacity", d => opacityScale(d.centrality));
         
         
         // Append labels to the groups
        node.append("text")
        .attr("dx", 12)
        .attr("dy", ".35em")
        .style("font-size", d => fontSizeScale(d.centrality) + "px")
        .text(function(d) {{
            return d.label;
        }});
         
         var link = svg.append("g")
         .attr("class", "links")
         .selectAll("path")
         .data(links)
         .enter().append("path")
         .attr("fill", "none")
         .attr("stroke", "#666")
         .attr("stroke-width", 0.8)
         .attr("stroke-opacity", 0.2);
         
         
         simulation
         .nodes(nodes)
         .on("tick", ticked);
         
         simulation.force("link")
         .links(links);
         
         
         
         // Add node labels
         
         
         // ... rest of the code ...
                     
         function ticked() {{
         node.attr("transform", d => `translate(${{d.x}},${{d.y}})`);
         
         link.attr("d", function(d) {{
         var dx = d.target.x - d.source.x,
             dy = d.target.y - d.source.y,
             dr = Math.sqrt(dx * dx + dy * dy); // distance between nodes
         return `M ${{d.source.x}} ${{d.source.y}} A ${{dr}} ${{dr}} 0 0 1 ${{d.target.x}} ${{d.target.y}}`;
         }});
         }}
         
         function dragstarted(d) {{
         if (!d3.event.active) simulation.alphaTarget(0.3).restart();
         d.fx = d.x;
         d.fy = d.y;
         }}
         
         function dragged(d) {{
         d.fx = d3.event.x;
         d.fy = d3.event.y;
         }}
         
         function dragended(d) {{
         if (!d3.event.active) simulation.alphaTarget(0);
         d.fx = null;
         d.fy = null;
         }}
         
      </script>
   </body>
</html>""")


In [170]:
with open('dagre_test.html', "w", encoding = "utf-8") as fp:
    fp.write("""<!DOCTYPE html>
<html lang="en">
   <head>
      <meta charset="UTF-8">
      <title>D3 Force Layout</title>
      <style>
         text {{
         font-family: sans-serif;
         }}
      </style>
   </head>
   <body>
      <svg width="1800" height="1600"></svg>
      <script src="https://d3js.org/d3.v5.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>


      <script>
      
import React, { useEffect, useRef } from 'react';
import * as d3 from 'd3';
import {dagStratify, sugiyama, decrossOpt} from 'd3-dag';

import PropTypes from 'prop-types';

const propTypes = {
  data: PropTypes.arrayOf(
    PropTypes.shape({
      id: PropTypes.string.isRequired,
      parentIds: PropTypes.arrayOf(PropTypes.string),
    }),
  ).isRequired,
};

const SugiyamaLayoutView = ({data}) => {
  const d3Chart = useRef();

  useEffect(() => {
    debugger;
    const dag = dagStratify()(data);
    
    const nodeRadius = 20;
    const layout = sugiyama() // base layout
      .decross(decrossOpt()) // minimize number of crossings
      .nodeSize((node) => [(node ? 3.6 : 0.25) * nodeRadius, 3 * nodeRadius]); // set node size instead of constraining to fit
    const { width, height } = layout(dag);

    // --------------------------------
    // This code only handles rendering
    // --------------------------------
    const svgSelection = d3.select(d3Chart.current);
    svgSelection.attr("viewBox", [0, 0, width, height].join(" "));
    const defs = svgSelection.append("defs"); // For gradients

    const steps = dag.size();
    const interp = d3.interpolateRainbow;
    const colorMap = new Map();
    //for (const [i, node] of dag.idescendants().entries()) {
    //for (const [i, node] of dag.idescendants()) {
    for (const [i, node] of [...dag].entries()) {
      try{
        colorMap.set(node.data.id, interp(i / steps));
      } catch (e){
        console.log("Error: ", i);
      }
    }

    // How to draw edges
    const line = d3
      .line()
      .curve(d3.curveCatmullRom)
      .x((d) => d.x)
      .y((d) => d.y);

    // Plot edges
    svgSelection
      .append("g")
      .selectAll("path")
      .data(dag.links())
      .enter()
      .append("path")
      .attr("d", ({ points }) => line(points))
      .attr("fill", "none")
      .attr("stroke-width", 3)
      .attr("stroke", ({ source, target }) => {
        // encodeURIComponents for spaces, hope id doesn't have a `--` in it
        const gradId = encodeURIComponent(`${source.data.id}--${target.data.id}`);
        const grad = defs
          .append("linearGradient")
          .attr("id", gradId)
          .attr("gradientUnits", "userSpaceOnUse")
          .attr("x1", source.x)
          .attr("x2", target.x)
          .attr("y1", source.y)
          .attr("y2", target.y);
        grad
          .append("stop")
          .attr("offset", "0%")
          .attr("stop-color", colorMap.get(source.data.id));
        grad
          .append("stop")
          .attr("offset", "100%")
          .attr("stop-color", colorMap.get(target.data.id));
        return `url(#${gradId})`;
      });

    // Select nodes
    const nodes = svgSelection
      .append("g")
      .selectAll("g")
      .data(dag.descendants())
      .enter()
      .append("g")
      .attr("transform", ({ x, y }) => `translate(${x}, ${y})`);

    // Plot node circles
    nodes
      .append("circle")
      .attr("r", nodeRadius)
      .attr("fill", (n) => colorMap.get(n.data.id));

    // Add text to nodes
    nodes
      .append("text")
      .text((d) => d.data.id)
      .attr("font-weight", "bold")
      .attr("font-family", "sans-serif")
      .attr("text-anchor", "middle")
      .attr("alignment-baseline", "middle")
      .attr("fill", "white");
  })

  return (
    <div id='d3demo'>
      <svg ref={d3Chart}></svg>
    </div>
  )
}

SugiyamaLayoutView.propTypes = propTypes;

export default SugiyamaLayoutView;
</script>
   </body>
</html>""")